In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [2]:
class ShowerEnv(Env):
    def __init__(self, shower_length):
        # Actions we can take: down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0.0]), high=np.array([100.0]))
        # Set start temp
        self.state = 38 + random.randint(-3, 3)
        # Set shower length
        self.shower_length = shower_length
    
    def step(self, action):
        
        # apply action
        # 0 -> -1
        # 1 -> 0
        # 2 -> 1
        self.state += (action - 1)
        
        # reduce shower length by 1 second
        self.shower_length -= 1
        
        # calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1;
        else:
            reward = -1
        # check if shower is done
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        # appply temperature noise, simulate the observations 
#         self.state += random.randint(-1, 1)
        self.state += (random.random() - 0.5) * 0.5
        
        # set palceholder for info 
        info = {}
        
        return self.state, reward, done, info 
        
        
    def render(self):
        # Implement viz
        pass
    def reset(self):
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        return self.state

In [3]:
env = ShowerEnv(shower_length = 60)

/usr/local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
env.action_space.sample()

1

In [5]:
env.observation_space.sample()

array([1.3573842], dtype=float32)

In [7]:
episodes = 10
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Socre:{}'.format(episode, score))

Episode:1 Socre:-60
Episode:2 Socre:-32
Episode:3 Socre:-54
Episode:4 Socre:-46
Episode:5 Socre:6
Episode:6 Socre:-54
Episode:7 Socre:-22
Episode:8 Socre:-12
Episode:9 Socre:-42
Episode:10 Socre:-4


In [8]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

def build_model(states, actions):
    model = Sequential()
#     model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [9]:
states = env.observation_space.shape
actions = env.action_space.n
print(states, actions)

(1,) 3


In [14]:
del model

In [15]:


model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [16]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 21:55 - reward: -1.0000

/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.8/site-packages/rl/memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 72s 7ms/step - reward: -0.7292
166 episodes - episode_reward: -43.711 [-60.000, 20.000] - loss: 2.048 - mae: 6.382 - mean_q: -4.472

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: -0.7772
167 episodes - episode_reward: -46.695 [-60.000, 4.000] - loss: 2.328 - mae: 10.869 - mean_q: -15.520

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 75s 8ms/step - reward: -0.7428
167 episodes - episode_reward: -44.539 [-60.000, 0.000] - loss: 2.960 - mae: 13.217 - mean_q: -19.062

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: -0.7534
166 episodes - episode_reward: -45.217 [-60.000, 12.000] - loss: 2.524 - mae: 12.688 - mean_q: -18.194

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 81s 8ms/step - reward: -0.7416
done, took 383.635 seconds


In [17]:
 scores = dqn.test(env, nb_episodes=20, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 20 episodes ...
Episode 1: reward: -56.000, steps: 60
Episode 2: reward: -58.000, steps: 60
Episode 3: reward: -56.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: -56.000, steps: 60
Episode 6: reward: -60.000, steps: 60
Episode 7: reward: -56.000, steps: 60
Episode 8: reward: -58.000, steps: 60
Episode 9: reward: -56.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -58.000, steps: 60
Episode 12: reward: -60.000, steps: 60
Episode 13: reward: -60.000, steps: 60
Episode 14: reward: -58.000, steps: 60
Episode 15: reward: -54.000, steps: 60
Episode 16: reward: -54.000, steps: 60
Episode 17: reward: -60.000, steps: 60
Episode 18: reward: -60.000, steps: 60
Episode 19: reward: -60.000, steps: 60
Episode 20: reward: -60.000, steps: 60
-58.0
